In [24]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from stores import get_vector_store

load_dotenv()

True

In [39]:
vector_store = get_vector_store()
llm = ChatOpenAI(model_name="gpt-5", api_key=os.getenv("OPENAI_API_KEY"))
retriever = vector_store.as_retriever(search_kwargs={"k": 2})
prompt = ChatPromptTemplate.from_template(
        """
        당신은 **형사법** 법령 검색 도움 봇입니다. 제공된 {context}를 바탕으로 사용자의 {question}에 답변해 주세요.
        
        **Required Rule**
            1. 반드시 답변의 근거가 된 참고자료({context})를 포함해야 합니다.
            2. {context}가 없을 경우 관련 법령을 검색할 수 없다고 답변하고 전문가의 도움을 받기를 제안하세요.
            3. **형사법**에 대한 답변만을 생성해야 합니다.
            4. 당신은 법령 정보만 제공합니다. 추가 의견은 추가하지 않습니다.

        Context:
        {context}

        Question:
        {question}
    """
)

def format_str(docs:list) -> str:
    return '\n'.join(doc.page_content for doc in docs)


chain = {
    "context": retriever | format_str,
    "question": RunnablePassthrough()
} | prompt | llm | StrOutputParser()

In [42]:
result = chain.invoke("내동생 내 돈 100만원 훔쳐갔는데 이거 신고하면 어떻게 되는지 궁금해")
print(result)

다음 법령에 따릅니다.

- 적용 죄명과 법정형
  - 절도: “타인의 재물을 절취한 자는 6년 이하의 징역 또는 1천만원 이하의 벌금에 처한다.”[형법 제329조]

- 친족 간 절도에 대한 특례(친고죄 해당)
  - “제328조의 규정은 제329조 내지 제332조의 죄 또는 미수범에 준용한다.”[형법 제344조]
  - “제323조의 죄를 지은 사람이 피해자의 친족인 경우에는 고소가 있어야 공소를 제기할 수 있다. … 피해자의 친족이 아닌 공범에 대해서는 제1항을 적용하지 아니한다.”(친족 범행에 대한 고소 요건 규정. 절도에는 제344조로 준용)[형법 제328조 제1항, 제3항]

정리
- 동생(친족)이 100만원을 가져간 경우, 죄명은 절도(형법 제329조)입니다.
- 다만 친족 간 범행에는 형법 제328조가 준용되므로, 피해자(질문자)의 고소가 있어야만 검사가 공소를 제기할 수 있습니다(친고죄). 고소가 없으면 공소 제기가 불가합니다. 친족이 아닌 공범이 있다면 그 공범에게는 친고요건이 적용되지 않습니다[형법 제344조, 제328조 제1항·제3항].
- 고소(신고)하면 수사가 진행되고, 검사가 공소를 제기할 수 있으며, 법정형 범위는 징역 6년 이하 또는 벌금 1천만원 이하입니다[형법 제329조].
